In [21]:
%%writefile cnt.input.lammps

variable T equal 300

units real
atom_style molecular
boundary f f f
pair_style lj/cut 14

bond_style harmonic
angle_style harmonic
dihedral_style opls
improper_style harmonic

special_bonds lj 0.0 0.0 0.5

read_data cnt1.data
include cnt.parm.lammps

group carbon_atoms type 1
variable carbon_xcm equal -1*xcm(carbon_atoms,x)
variable carbon_ycm equal -1*xcm(carbon_atoms,y)
variable carbon_zcm equal -1*xcm(carbon_atoms,z)
displace_atoms carbon_atoms move ${carbon_xcm} ${carbon_ycm} ${carbon_zcm}

change_box all x final -40 40 y final -40 40 z final -40 40

variable zmax equal bound(carbon_atoms,zmax)-0.5
variable zmin equal bound(carbon_atoms,zmin)+0.5
region rtop block INF INF INF INF ${zmax} INF
region rbot block INF INF INF INF INF ${zmin}
region rmid block INF INF INF INF ${zmin} ${zmax}

group carbon_top region rtop
group carbon_bot region rbot
group carbon_mid region rmid

variable zmax_del equal ${zmax}-2
variable zmin_del equal ${zmin}+2
region rdel block INF INF INF INF ${zmin_del} ${zmax_del}
group rdel region rdel
delete_atoms random fraction 0.03 no rdel NULL 242432 bond yes

reset_atoms id sort yes
velocity carbon_mid create ${T} 428732 mom yes rot yes
fix mynve all nve
compute Tmid carbon_mid temp
fix myber carbon_mid temp/berendsen ${T} ${T} 100
fix_modify myber temp Tmid

fix mysf1 carbon_top setforce 0 0 0
fix mysf2 carbon_bot setforce 0 0 0 
velocity carbon_top set 0 0 0
velocity carbon_bot set 0 0 0

variable L equal xcm(carbon_top,z)-xcm(carbon_bot,z)
fix at1 all ave/time 10 10 100 v_L file output_cnt_length.dat
fix at2 all ave/time 10 10 100 f_mysf1[1] f_mysf2[1] file output_edge_force.dat

dump mydump all atom 1000 dump.lammpstrj
fix at3 all ave/time 10 10 100 c_Tmid file output_temperature_middle_group.dat

thermo 100
thermo_modify temp Tmid

timestep 1.0
run 5000

velocity carbon_top set 0 0 0.0005
velocity carbon_bot set 0 0 -0.0005
run 10000

Overwriting cnt.input.lammps


In [22]:
%%writefile cnt.parm.lammps

pair_coeff 1 1 .066 3.4
bond_coeff 1 469 1.4
angle_coeff 1 63 120
dihedral_coeff 1 0 7.25 0 0
improper_coeff 1 5 180

Overwriting cnt.parm.lammps


In [23]:
! lmp -in cnt.input.lammps

LAMMPS (27 Jun 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\Users\aravs\AppData\Local\LAMMPS 64-bit 27Jun2024\plugins
Reading data file ...
  orthogonal box = (-40 -40 -12.130411) to (40 40 67.869589)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  700 atoms
  scanning bonds ...
  3 = max bonds/atom
  scanning angles ...
  3 = max angles/atom
  scanning dihedrals ...
  12 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  orthogonal box = (-40 -40 -12.130411) to (40 40 67.869589)
  1 by 1 by 1 MPI processor grid
  reading bonds ...
  1035 bonds
  reading angles ...
  2040 angles
  reading dihedrals ...
  4030 dihedrals
  reading impropers ...
  670 impropers
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0.5     
  special bond factors coul:  0        0        0       
     3 = max # of 1-2 neighbors
     6 = max # o

In [59]:
%%writefile cnt.input2.lammps

variable T equal 300

units metal
atom_style atomic
boundary  p p p
pair_style airebo 2.5 1 1 

# bond_style harmonic
# angle_style harmonic
# dihedral_style opls
# improper_style harmonic

# special_bonds lj 0.0 0.0 0.5
read_data cnt_atom.data
pair_coeff * * CH.airebo C 

group carbon_atoms type 1
variable carbon_xcm equal -1*xcm(carbon_atoms,x)
variable carbon_ycm equal -1*xcm(carbon_atoms,y)
variable carbon_zcm equal -1*xcm(carbon_atoms,z)
displace_atoms carbon_atoms move ${carbon_xcm} ${carbon_ycm} ${carbon_zcm}

change_box all x final -40 40 y final -40 40 z final -40 40

variable zmax equal bound(carbon_atoms,zmax)-0.5
variable zmin equal bound(carbon_atoms,zmin)+0.5
region rtop block INF INF INF INF ${zmax} INF
region rbot block INF INF INF INF INF ${zmin}
region rmid block INF INF INF INF ${zmin} ${zmax}

group carbon_top region rtop
group carbon_bot region rbot
group carbon_mid region rmid

variable zmax_del equal ${zmax}-2
variable zmin_del equal ${zmin}+2
region rdel block INF INF INF INF ${zmin_del} ${zmax_del}
group rdel region rdel
delete_atoms random fraction 0.02 no rdel NULL 242342

reset_atoms id sort yes
velocity carbon_mid create ${T} 28424 mom yes rot yes
fix mynve all nve
compute Tmid carbon_mid temp
fix myber carbon_mid temp/berendsen ${T} ${T} 0.1
fix_modify myber temp Tmid

fix mysf1 carbon_bot setforce 0 0 0
fix mysf2 carbon_top setforce 0 0 0
velocity carbon_bot set 0 0 0
velocity carbon_top set 0 0 0

variable L equal xcm(carbon_top,z)-xcm(carbon_bot,z)
fix at1 all ave/time 10 10 100 v_L file output_cnt_length.dat
fix at2 all ave/time 10 10 100 f_mysf1[1] f_mysf2[1] file outputedgeforce.dat

dump mydump all atom 1000 dump2.lammpstrj

thermo 100
thermo_modify temp Tmid

timestep 0.0005
# run 5000

velocity carbon_top set 0 0 0.15
run 280000

Overwriting cnt.input2.lammps


In [60]:
! lmp -in cnt.input2.lammps

LAMMPS (27 Jun 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\Users\aravs\AppData\Local\LAMMPS 64-bit 27Jun2024\plugins
Reading data file ...
  orthogonal box = (-40 -40 -12.130411) to (40 40 67.869589)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  700 atoms
  read_data CPU = 0.002 seconds
Reading airebo potential file CH.airebo with DATE: 2011-10-25
700 atoms in group carbon_atoms
Displacing atoms ...
Changing box ...
  orthogonal box = (-40 -40 -12.130411) to (40 40 67.869589)
  orthogonal box = (-40 -40 -12.130411) to (40 40 67.869589)
  orthogonal box = (-40 -40 -40) to (40 40 40)
10 atoms in group carbon_top
10 atoms in group carbon_bot
680 atoms in group carbon_mid
630 atoms in group rdel
Deleted 9 atoms, new total = 691
Resetting atom IDs ...
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000


In [75]:
%%writefile supposedlysimpleinput.lammps

variable T equal 300

units real
atom_style molecular
boundary f f f
pair_style lj/cut 14

bond_style harmonic
angle_style harmonic
dihedral_style opls
improper_style harmonic

special_bonds lj 0.0 0.0 0.5

read_data cnt_molecular.data
include parm.lammps

group carbon_atoms type 1

fix mynve all nve
fix myber all temp/berendsen ${T} ${T} 100
fix myrct all recenter 0 0 0

thermo 1000
dump mydmp all atom 1000 dump1.lammpstrj

timestep 1.0
run 10000

Overwriting supposedlysimpleinput.lammps


In [76]:
! lmp -in supposedlysimpleinput.lammps

LAMMPS (27 Jun 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\Users\aravs\AppData\Local\LAMMPS 64-bit 27Jun2024\plugins
Reading data file ...
  orthogonal box = (-40 -40 -40) to (40 40 40)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  690 atoms
  reading velocities ...
  690 velocities
  scanning bonds ...
  3 = max bonds/atom
  scanning angles ...
  3 = max angles/atom
  scanning dihedrals ...
  12 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  orthogonal box = (-40 -40 -40) to (40 40 40)
  1 by 1 by 1 MPI processor grid
  reading bonds ...
  1005 bonds
  reading angles ...
  1950 angles
  reading dihedrals ...
  3791 dihedrals
  reading impropers ...
  630 impropers
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0.5     
  special bond factors coul:  0        0        0       
     3 = max # of 1-2 neighbors
  

In [87]:
%%writefile breakable.data.cnt.input.lammps

variable T equal 300

units metal 
atom_style atomic
boundary p p p
pair_style airebo 2.5 1 1

read_data cnt_atom.data
pair_coeff * * CH.airebo C

change_box all x final -60 60 y final -60 60 z final -60 60 

group carbon_atoms type 1
variable carbon_xcm equal -1*xcm(carbon_atoms,x)
variable carbon_ycm equal -1*xcm(carbon_atoms,y)
variable carbon_zcm equal -1*xcm(carbon_atoms,z)
displace_atoms carbon_atoms move ${carbon_xcm} ${carbon_ycm} ${carbon_zcm}

variable zmax equal bound(carbon_atoms,zmax)-0.5
variable zmin equal bound(carbon_atoms,zmin)+0.5
region rtop block INF INF INF INF ${zmax} INF
region rbot block INF INF INF INF INF ${zmin}
region rmid block INF INF INF INF ${zmin} ${zmax}

group carbon_top region rtop
group carbon_bot region rbot
group carbon_mid region rmid

velocity carbon_mid create ${T} 42748 mom yes rot yes
fix mynve all nve 
compute Tmid carbon_mid temp
fix myber carbon_mid temp/berendsen ${T} ${T} 0.1
fix_modify myber temp Tmid

fix mysf1 carbon_top setforce 0 0 NULL
fix mysf2 carbon_bot setforce 0 0 NULL
velocity carbon_top set 0 0 NULL
velocity carbon_bot set 0 0 NULL

dump mydump all atom 1000 cnt.breakable.dump.lammpstrj

thermo 100
thermo_modify temp Tmid

timestep 0.0005
run 5000

variable L equal xcm(carbon_top,z)-xcm(carbon_bot,z)
variable strain equal 100*(v_L-${L})/${L} 
fix at1 all ave/time 20 100 2000 v_strain file cnt_strain_output.dat

variable eV_to_joule equal 1.60218e-19
variable Atom equal 1e-10
variable pi equal 3.14159
variable f_total_eV equal f_mysf2[3]-f_mysf1[3]
variable f_total_newton equal v_f_total_eV*${eV_to_joule}/${Atom}
variable rad equal 5.2e-10 
variable dC equal 3.4e-10
variable A equal ${pi}*${rad}*${dC}
variable stress equal v_f_total_newton/${A}

fix at2 all ave/time 20 100 2000 v_stress file cnt_stress_output.dat

fix mysf1 carbon_top setforce 0 0 0
fix mysf2 carbon_bot setforce 0 0 0

velocity carbon_top set 0 0 0.05
velocity carbon_bot set 0 0 -0.05

run 40000

Overwriting breakable.data.cnt.input.lammps


In [88]:
!lmp -in breakable.data.cnt.input.lammps

LAMMPS (27 Jun 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\Users\aravs\AppData\Local\LAMMPS 64-bit 27Jun2024\plugins
Reading data file ...
  orthogonal box = (-40 -40 -12.130411) to (40 40 67.869589)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  700 atoms
  read_data CPU = 0.003 seconds
Reading airebo potential file CH.airebo with DATE: 2011-10-25
Changing box ...
  orthogonal box = (-60 -40 -12.130411) to (60 40 67.869589)
  orthogonal box = (-60 -60 -12.130411) to (60 60 67.869589)
  orthogonal box = (-60 -60 -60) to (60 60 60)
700 atoms in group carbon_atoms
Displacing atoms ...
10 atoms in group carbon_top
10 atoms in group carbon_bot
680 atoms in group carbon_mid

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log fi